# 1. 빅벨류-산단정보-등록공장 데이터 가져오기

In [18]:
# 산단정보 - 등록공장 데이터 가져오기

import pandas as pd
bigvalue  = pd.read_csv('../CompetitionData/빅밸류/IND_REGINFO_202105.csv', encoding = 'cp949', sep = '|')
bigvalue.fillna('알수없음', inplace=True) # 널값 채우기
bigvalue.sample()

,REG_ID,IC_ADMIN,FIRM_NAME,IC_NAME,IC_CLASS,REG_DATE,TEL,EMP_NUM,PRODUCTS,IND_SIZE,...,POLL_WAT,POLL_NOI,LIV_WAT,IND_WAT,WEB,IND_TYPE,IND_CODE,ADDR,CAP_IN,CAP_OUT
39485,10188196,경상남도 진주시,이룸P&P,알수없음,일반,2.01605e+07,055-747-7970,7,골판지상자,소기업,...,해당없음,무,0.5,0,알수없음,골판지 제조업 외 1 종,"17211, 17212",경상남도 진주시 대곡면 대화로 53-6,0,0


In [19]:
# 서울 내에 있는 기업 데이터를 가져오는 것이므로 위치가 서울인 데이터만 고르기
seoul = bigvalue[bigvalue['IC_ADMIN'].str.contains('서울')]
seoul.sample()

,REG_ID,IC_ADMIN,FIRM_NAME,IC_NAME,IC_CLASS,REG_DATE,TEL,EMP_NUM,PRODUCTS,IND_SIZE,...,POLL_WAT,POLL_NOI,LIV_WAT,IND_WAT,WEB,IND_TYPE,IND_CODE,ADDR,CAP_IN,CAP_OUT
71895,10007035,한국산업단지공단 서울지역본부,(주)성호라인,한국수출(서울디지털),국가산업단지,2.01411e+07,02-3408-0515,25,수령증 인쇄 및 카드배송프로그램(어플리케이션 S/W) 개발,소기업,...,알수없음,알수없음,알수없음,알수없음,알수없음,기타 인쇄업 외 2 종,"18113, 18119, 58222","서울특별시 금천구 디지털로9길 56, 8층 804호 (가산동, 코오롱테크노밸리) 외...",알수없음,알수없음


In [20]:
# 서울시구만 고르기
seoul_select = seoul[['IC_ADMIN', 'IND_TYPE', 'IND_CODE', 'ADDR']]
seoul_select['서울시구']= seoul_select['ADDR'].apply(lambda x :str(x)[:9]) 
seoul_select.reset_index(inplace=True, drop=True)
seoul_select.sample()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,IC_ADMIN,IND_TYPE,IND_CODE,ADDR,서울시구
11400,한국산업단지공단 서울지역본부,방송장비 제조업 외 1 종,"26421, 58222","서울특별시 금천구 가산디지털2로 98 (가산동, 아이티캐슬) (총 4 필지) 1동 ...",서울특별시 금천구


In [21]:
# 같은 기업이라도 업종코드(IND_CODE)가 여러개인 기업이 있다.
# 이 경우 해당 기업 개수 늘려서 업종 코드 분리하기
for i in range(len(seoul_select)):
    seoul_select['IND_CODE'][i] = list(seoul_select['IND_CODE'][i].split(', '))

# explode로 늘리기
seoul_select_explode = seoul_select.explode('IND_CODE')
seoul_select_explode.head(3)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3345: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


,IC_ADMIN,IND_TYPE,IND_CODE,ADDR,서울시구
0,서울특별시 종로구,경 인쇄업 외 2 종,18111,서울특별시 종로구 자하문로16길 8 (창성동),서울특별시 종로구
0,서울특별시 종로구,경 인쇄업 외 2 종,18113,서울특별시 종로구 자하문로16길 8 (창성동),서울특별시 종로구
0,서울특별시 종로구,경 인쇄업 외 2 종,18119,서울특별시 종로구 자하문로16길 8 (창성동),서울특별시 종로구


# 2. 업종 코드 정리하기 - 고용노동부_업종코드
- 출처: https://www.data.go.kr/data/15049592/fileData.do

In [23]:
import numpy as np

# 고용노동부_업종코드 데이터 불러오기
Industry_codes = pd.read_csv('../김은선/고용노동부_업종코드_20201006.csv', encoding='cp949')
Industry_codes.sample()

,대분류,중분류,소분류,업종코드,업종명
1335,G,45.0,452.0,45212,자동차용 전용 신품 부품 판매업


In [24]:
# 무량공처 팀에서 정한 업종 코드 타입 지정하기.
# 노션/Datasets: 2019년귀속업종분류코드책자 참고.

types = ['G', 'L', 'M', 'N', 'K', 'P', 'J']

Industry_selected = Industry_codes[Industry_codes['대분류'].isin(types)]
idx = Industry_selected[Industry_selected['대분류'] == 'G'].index
Industry_selected.loc[idx, '업종'] = '도매 및 소매업'

idx = Industry_selected[Industry_selected['대분류'] == 'L'].index
Industry_selected.loc[idx, '업종'] = '부동산업'

idx = Industry_selected[Industry_selected['대분류'] == 'M'].index
Industry_selected.loc[idx, '업종'] = '전문, 과학 및 기술 서비스업'

idx = Industry_selected[Industry_selected['대분류'] == 'N'].index
Industry_selected.loc[idx, '업종'] = '사업시설 관리, 사업지원 및 임대 서비스업'

idx = Industry_selected[Industry_selected['대분류'] == 'K'].index
Industry_selected.loc[idx, '업종'] = '금융 및 보험업'

idx = Industry_selected[Industry_selected['대분류'] == 'P'].index
Industry_selected.loc[idx, '업종'] = '교육 서비스업'

idx = Industry_selected[Industry_selected['대분류'] == 'J'].index
Industry_selected.loc[idx, '업종'] = '정보통신업'

/home/jovyan/.local/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/jovyan/.local/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [25]:
Industry_selected.sample()

,대분류,중분류,소분류,업종코드,업종명,업종
1928,M,71.0,715.0,7153,경영 컨설팅 및 공공 관계 서비스업,"전문, 과학 및 기술 서비스업"


# 3. 빅벨류 데이터에서의 업종코드 정리하기 (위 고용노동부 데이터 참고)

In [49]:
seoul_select_explode['organized_업종명'] = '알수없음' # 정리된 업종명 칼럼 지정하기

merged_seoul = pd.merge(seoul_select_explode, Industry_selected, how='left', left_on='IND_CODE', right_on='업종코드')
merged_seoul = merged_seoul.dropna()
merged_seoul.reset_index(inplace=True, drop=True)

merged_seoul = merged_seoul[['IC_ADMIN', 'ADDR', '서울시구', '업종']] # 필요한 칼럼만 가져오기
merged_seoul.head(3)

,IC_ADMIN,ADDR,서울시구,업종
0,서울특별시 종로구,서울특별시 종로구 자하문로1길 41 (필운동),서울특별시 종로구,정보통신업
1,서울특별시 종로구,"서울특별시 종로구 수표로26길 12, 405호 (낙원동, 파고다빌딩)",서울특별시 종로구,도매 및 소매업
2,서울특별시 종로구,서울특별시 종로구 통일로16길 5 (무악동),서울특별시 종로구,정보통신업


In [50]:
# 업종별 개수 확인
merged_seoul['업종'].value_counts()

정보통신업                      609
전문, 과학 및 기술 서비스업           265
도매 및 소매업                    35
부동산업                        27
사업시설 관리, 사업지원 및 임대 서비스업      9
Name: 업종, dtype: int64

# 4. 해당 기업 데이터에 위도/경도 붙이기

In [51]:
# 확실히 하기 위해 '서울'과 중복 데이터 제거
merged_seoul = merged_seoul[merged_seoul['서울시구'].str.contains('서울')]
merged_seoul = merged_seoul.drop_duplicates()
merged_seoul.reset_index(inplace=True, drop=True)
merged_seoul.head(3)

,IC_ADMIN,ADDR,서울시구,업종
0,서울특별시 종로구,서울특별시 종로구 자하문로1길 41 (필운동),서울특별시 종로구,정보통신업
1,서울특별시 종로구,"서울특별시 종로구 수표로26길 12, 405호 (낙원동, 파고다빌딩)",서울특별시 종로구,도매 및 소매업
2,서울특별시 종로구,서울특별시 종로구 통일로16길 5 (무악동),서울특별시 종로구,정보통신업


In [52]:
# ADDR로부터 동 추출하기

merged_seoul['동'] = '알수없음'

for i in range(len(merged_seoul)):
    
    text = merged_seoul['ADDR'][i]
    try:
        idx1 = text.index('(')
        idx2 = text.index(')')
        text2 = text[idx1+1:idx2]
    except:
        text2 = text
    
    try:
        idx3 = text2.index(',')
        text2 = text2[:idx3]
    except:
        text2 = text2
    
    merged_seoul['동'][i] = text2

In [53]:
# 구 제거하기

def cleansing_gu(text):
    i = text.rfind('구 ')
    return text[i+1:]


for i in range(len(merged_seoul)):
    merged_seoul[i:i+1]['ADDR']=cleansing_gu(merged_seoul.iloc[i]['ADDR']).rstrip()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [54]:
# 구와 동을 제거한 구체적인 주소명만 남기기

def cleansing_dong(text):
    try:
        idx = text.index(',')
        text = text[:idx]
    except:
        pass
    try:
        idx = text.index('(')
        text = text[:idx]
    except:
        pass
    
    return text


for i in range(len(merged_seoul)):
    merged_seoul[i:i+1]['ADDR']=cleansing_dong(merged_seoul.iloc[i]['ADDR']).rstrip()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [55]:
merged_seoul.head(3)

,IC_ADMIN,ADDR,서울시구,업종,동
0,서울특별시 종로구,자하문로1길 41,서울특별시 종로구,정보통신업,필운동
1,서울특별시 종로구,수표로26길 12,서울특별시 종로구,도매 및 소매업,낙원동
2,서울특별시 종로구,통일로16길 5,서울특별시 종로구,정보통신업,무악동


In [56]:
# 필요한 위도경도 데이터 불러오기
geo=pd.read_excel('/home/jovyan/WIDE_HOME//유수빈/add.xlsx')
geo.drop('Unnamed: 0', axis=1, inplace=True)
geo.head(3)

,주소,lon,lat
0,서울특별시 종로구 청운동 자하문로 94 0,126.970406,37.584415
1,서울특별시 종로구 청운동 자하문로 96 0,126.970423,37.584513
2,서울특별시 종로구 청운동 자하문로 98 0,126.970394,37.584697


In [57]:
# 위 주소에 해당이 되는 건물들의 위도와 경도 가져와서 합치기

from tqdm import tqdm

merged_seoul['lon'] = 0.0
merged_seoul['lat'] = 0.0

for i in tqdm(range(len(merged_seoul))):
    try:
        idx = geo[geo['주소'].str.contains(merged_seoul['ADDR'][i])].index
        lon = geo.iloc[idx]['lon'].values[0]
        lat = geo.iloc[idx]['lat'].values[0]
    except:
        lon = 0.0
        lat = 0.0
    
    merged_seoul.loc[i, 'lon'] = lon
    merged_seoul.loc[i, 'lat'] = lat

100%|██████████| 768/768 [02:53<00:00,  4.57it/s]


In [58]:
# 동도로명과 구 칼럼 정리
merged_seoul['동도로명'] = merged_seoul['동']+ ' ' + merged_seoul['ADDR']

merged_seoul['구'] = '알수없음'
for i in range(len(merged_seoul)):
    merged_seoul['구'][i] = merged_seoul['서울시구'][i][6:]
    
merged_seoul.drop('IC_ADMIN', axis=1, inplace=True)

# 필요한 칼럼만 가져오기
merged_seoul = merged_seoul[['업종', 'ADDR', '구', '동', '동도로명', '서울시구', 'lon', 'lat']]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [96]:
seoul_bigvalue_geo_data = merged_seoul.copy()
seoul_bigvalue_geo_data.head(3)

,업종,ADDR,구,동,동도로명,서울시구,lon,lat
0,정보통신업,자하문로1길 41,종로구,필운동,필운동 자하문로1길 41,서울특별시 종로구,126.970063,37.576642
1,도매 및 소매업,수표로26길 12,종로구,낙원동,낙원동 수표로26길 12,서울특별시 종로구,126.989733,37.572100
2,정보통신업,통일로16길 5,종로구,무악동,무악동 통일로16길 5,서울특별시 종로구,126.957166,37.575758


In [97]:
# 위경도가 맵핑 되지 못한 데이터 보기
check = seoul_bigvalue_geo_data[seoul_bigvalue_geo_data['lon'] == 0.0]
print(len(check)) # 총 38개 데이터
check.head()

38


,업종,ADDR,구,동,동도로명,서울시구,lon,lat
5,정보통신업,퇴계로 253-6,중구,충무로5가,충무로5가 퇴계로 253-6,서울특별시 중구,0.0,0.0
7,"전문, 과학 및 기술 서비스업",퇴계로50길 26-6,중구,묵정동,묵정동 퇴계로50길 26-6,서울특별시 중구,0.0,0.0
9,정보통신업,을지로40길 21-13,중구,을지로6가,을지로6가 을지로40길 21-13,서울특별시 중구,0.0,0.0
10,정보통신업,을지로14길 16-8,중구,을지로3가,을지로3가 을지로14길 16-8,서울특별시 중구,0.0,0.0
28,"전문, 과학 및 기술 서비스업",도봉로136다길 26-1,도봉구,창동,창동 도봉로136다길 26-1,서울특별시 도봉구,0.0,0.0


In [98]:
check['ADDR'].values

array([' 퇴계로 253-6', ' 퇴계로50길 26-6', ' 을지로40길 21-13', ' 을지로14길 16-8',
       ' 도봉로136다길 26-1', ' 동일로186길 53-50', ' 이화여대2가길 16-5',
       ' 남부순환로64길 27-5', ' 구로동 187-10번지 코오롱싸이언스밸리1차 B1층 B103호',
       ' 구로동 212-13번지 벽산디지털밸리III 7층 710-712호', ' 디지털로31길 38-9',
       ' 디지털로31길 38-21', ' 디지털로31길 38-21', ' 디지털로31길 38-21',
       ' 디지털로31길 38-9', ' 디지털로31길 38-9', ' 구로동 235번지 한신아이티타워 6층 9호',
       ' 구로동 222-31번지 13층 1304', ' 구로동 197-28번지 이앤씨벤처드림타워6차 3층 5호',
       ' 디지털로31길 38-9', ' 디지털로31길 38-21',
       ' 가산동 470-8번지 케이씨씨 웰츠밸리 5층 505호', ' 가산동 319번지 호서대벤처타워 12층 3호',
       ' 가산디지털1로 33-33', ' 가산동 448번지 대륭테크노타운Ⅲ 6층 609호', ' 디지털로 119',
       ' 가산동 371-50번지 에이스하이엔드타워3 4층 405', ' 가산디지털1로 205-28',
       ' 가산동 371-37번지 STX V-TOWER 7층 708호', ' 디지털로10길 77',
       ' 가산동 60-69번지 엘리시아 11층 1103', ' 경인로82길 3-4', ' 경인로82길 3-4',
       ' 도림로125가길 6-1', ' 선유로 108-7', ' 신사동 506번지 4층', ' 신사동 506번지 6층',
       ' 자하문로2길 13-3'], dtype=object)

In [99]:
geo.head()['주소'].values

array(['서울특별시 종로구 청운동 자하문로 94 0', '서울특별시 종로구 청운동 자하문로 96 0',
       '서울특별시 종로구 청운동 자하문로 98 0', '서울특별시 종로구 청운동 자하문로 99 3',
       '서울특별시 종로구 청운동 자하문로 99 4'], dtype=object)

- 위경도가 있는 데이터를 보면 '자하문로 94 0' 같은 구조이다. 맵핑 되지 못한 주소 데이터는 대부분 '디지털로31길 38-9'처럼 '-'가 있어서 맵핑되지 못한 경우가 많으므로 해당 부분은 이를 지워 다시 맵핑하고 이외는 구글리을 통해 위도경도를 입력한다.

In [100]:
# 1. 수기로 고치기
idx = 5
seoul_bigvalue_geo_data.loc[idx, 'lat'] = '37.563389'
seoul_bigvalue_geo_data.loc[idx, 'lon'] = '127.00032'

idx = 7
seoul_bigvalue_geo_data.loc[idx, 'lat'] = '37.561616'
seoul_bigvalue_geo_data.loc[idx, 'lon'] = '127.00062'

In [101]:
# 2. 위경도가 0인부분 다시 맵핑하기
check['ADDR'] = check['ADDR'].str.replace('-', ' ')

check.reset_index(inplace=True, drop=False)

for i in tqdm(range(len(check))):
    try:
        idx = geo[geo['주소'].str.contains(check['ADDR'][i])].index
        lon = geo.iloc[idx]['lon'].values[0]
        lat = geo.iloc[idx]['lat'].values[0]
    except:
        lon = 0.0
        lat = 0.0
    
    check.loc[i, 'lon'] = lon
    check.loc[i, 'lat'] = lat
    
    
# 위경도가 0이었지만 새로 맵핑한 데이터와 기존 데이터 합치기
for i in range(len(seoul_bigvalue_geo_data)):
    if i in (check['index'].values):
        seoul_bigvalue_geo_data.loc[i, 'lon'] = check[check['index'] == i]['lon'].values[0]
        seoul_bigvalue_geo_data.loc[i, 'lat'] = check[check['index'] == i]['lat'].values[0]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
  0%|          | 0/38 [00:00<?, ?it/s]/home/jovyan/.local/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
100%|██████████| 38/38 [00:08<00:00,  4.25it/s]


In [102]:
# 3. 수기로 고치기
idx = 51
seoul_bigvalue_geo_data.loc[idx, '동'] = '구로동'
seoul_bigvalue_geo_data.loc[idx, 'lat'] = '37.485118'
seoul_bigvalue_geo_data.loc[idx, 'lon'] = '126.89882'

idx = 53
seoul_bigvalue_geo_data.loc[idx, '동'] = '구로동'
seoul_bigvalue_geo_data.loc[idx, 'lat'] = '37.483137'
seoul_bigvalue_geo_data.loc[idx, 'lon'] = '126.89372'

idx = 169
seoul_bigvalue_geo_data.loc[idx, '동'] = '구로동'
seoul_bigvalue_geo_data.loc[idx, 'lat'] = '37.480021'
seoul_bigvalue_geo_data.loc[idx, 'lon'] = '126.88617'

idx = 182
seoul_bigvalue_geo_data.loc[idx, '동'] = '구로동'
seoul_bigvalue_geo_data.loc[idx, 'lat'] = '37.482975'
seoul_bigvalue_geo_data.loc[idx, 'lon'] = '126.897806'

idx = 185
seoul_bigvalue_geo_data.loc[idx, '동'] = '구로동'
seoul_bigvalue_geo_data.loc[idx, 'lat'] = '37.486586'
seoul_bigvalue_geo_data.loc[idx, 'lon'] = '126.89365'

idx = 293
seoul_bigvalue_geo_data.loc[idx, '동'] = '가산동'
seoul_bigvalue_geo_data.loc[idx, 'lat'] = '37.472241'
seoul_bigvalue_geo_data.loc[idx, 'lon'] = '126.88593'

idx = 363
seoul_bigvalue_geo_data.loc[idx, '동'] = '가산동'
seoul_bigvalue_geo_data.loc[idx, 'lat'] = '37.478227'
seoul_bigvalue_geo_data.loc[idx, 'lon'] = '126.87919'

idx = 415
seoul_bigvalue_geo_data.loc[idx, '동'] = '가산동'
seoul_bigvalue_geo_data.loc[idx, 'lat'] = '37.561616'
seoul_bigvalue_geo_data.loc[idx, 'lon'] = '127.00062'

idx = 491
seoul_bigvalue_geo_data.loc[idx, '동'] = '가산동'
seoul_bigvalue_geo_data.loc[idx, 'lat'] = '37.476642'
seoul_bigvalue_geo_data.loc[idx, 'lon'] = '126.88414'

idx = 548
seoul_bigvalue_geo_data.loc[idx, '동'] = '가산동'
seoul_bigvalue_geo_data.loc[idx, 'lat'] = '37.477129'
seoul_bigvalue_geo_data.loc[idx, 'lon'] = '126.88375'

idx = 581
seoul_bigvalue_geo_data.loc[idx, '동'] = '가산동'
seoul_bigvalue_geo_data.loc[idx, 'lat'] = '37.475604'
seoul_bigvalue_geo_data.loc[idx, 'lon'] = '126.89080'

idx = 672
seoul_bigvalue_geo_data.loc[idx, '동'] = '가산동'
seoul_bigvalue_geo_data.loc[idx, 'lat'] = '37.477878'
seoul_bigvalue_geo_data.loc[idx, 'lon'] = '126.88600'

idx = 712
seoul_bigvalue_geo_data.loc[idx, '동'] = '신사동'
seoul_bigvalue_geo_data.loc[idx, 'lat'] = '37.526323'
seoul_bigvalue_geo_data.loc[idx, 'lon'] = '127.03715'

idx = 713
seoul_bigvalue_geo_data.loc[idx, '동'] = '신사동'
seoul_bigvalue_geo_data.loc[idx, 'lat'] = '37.526323'
seoul_bigvalue_geo_data.loc[idx, 'lon'] = '127.03715'

idx = seoul_bigvalue_geo_data[seoul_bigvalue_geo_data['동도로명'].str.contains('케이씨씨 웰츠밸리 5층 505호')].index
seoul_bigvalue_geo_data.loc[idx, '동'] = '가산동'
seoul_bigvalue_geo_data.loc[idx, 'lon'] = '126.879732'
seoul_bigvalue_geo_data.loc[idx, 'lat'] = '37.483727'

In [103]:
# 확인
seoul_bigvalue_geo_data[seoul_bigvalue_geo_data['lon'] == 0.0]

,업종,ADDR,구,동,동도로명,서울시구,lon,lat


In [104]:
# 잘못된 동 이름 고치기

idx = seoul_bigvalue_geo_data[seoul_bigvalue_geo_data['동'].str.contains('목2동')]['동'].index
seoul_bigvalue_geo_data.loc[idx, '동'] = '목2동'

idx = seoul_bigvalue_geo_data[seoul_bigvalue_geo_data['동'].str.contains('301-2~301-7호')].index
seoul_bigvalue_geo_data.loc[idx, '동'] = '가산동'

idx = seoul_bigvalue_geo_data[seoul_bigvalue_geo_data['동'].str.contains('일부')].index
seoul_bigvalue_geo_data.loc[idx, '동'] = '구로동'

idx = seoul_bigvalue_geo_data[seoul_bigvalue_geo_data['동'].str.contains('임차')].index
seoul_bigvalue_geo_data.loc[idx, '동'] = '구로동'

idx = seoul_bigvalue_geo_data[seoul_bigvalue_geo_data['동'].str.contains('자가')].index
seoul_bigvalue_geo_data.loc[idx, '동'] = '가산동'

idx = seoul_bigvalue_geo_data[seoul_bigvalue_geo_data['동'].str.contains('신정3동1254')].index
seoul_bigvalue_geo_data.loc[idx, '동'] = '신정동'

In [106]:
seoul_bigvalue_geo_data.head()

,업종,ADDR,구,동,동도로명,서울시구,lon,lat
0,정보통신업,자하문로1길 41,종로구,필운동,필운동 자하문로1길 41,서울특별시 종로구,126.97,37.5766
1,도매 및 소매업,수표로26길 12,종로구,낙원동,낙원동 수표로26길 12,서울특별시 종로구,126.99,37.5721
2,정보통신업,통일로16길 5,종로구,무악동,무악동 통일로16길 5,서울특별시 종로구,126.957,37.5758
3,정보통신업,세종대로 124,중구,태평로1가,태평로1가 세종대로 124,서울특별시 중구,126.978,37.5673
4,정보통신업,삼일대로6길 16,중구,충무로2가,충무로2가 삼일대로6길 16,서울특별시 중구,126.99,37.5624


In [105]:
#'서울_빅벨류_IND_REGINFO_202105_주소 및 경위도.csv'로 저장

#seoul_bigvalue_geo_data.to_csv('../서울_빅벨류_IND_REGINFO_202105_주소 및 경위도.csv', encoding='utf-8-sig')

In [114]:
seoul_bigvalue_geo_data.shape

(768, 8)

# 5. 더존비즈온 기업 데이터와 합치기

In [110]:
# 데이터 불러오기
thezone  = pd.read_csv('../서울_더존비즈온기업_주소 및 경위도(detail).csv')
thezone.drop('Unnamed: 0', axis=1, inplace=True)

# 필요한 칼럼만 추출
thezone = thezone[['표준 산업분류코드 대분류', 'ADDR', '구', '동', '동도로명', '서울시구', 'lon_mean_detail', 'lat_mean_detail']]
thezone.head()

,표준 산업분류코드 대분류,ADDR,구,동,동도로명,서울시구,lon_mean_detail,lat_mean_detail
0,도매 및 소매업,퇴계로75길,중구,흥인동,흥인동퇴계로75길,서울특별시 중구,127.015582,37.566224
1,"전문, 과학 및 기술 서비스업",퇴계로75길,중구,흥인동,흥인동퇴계로75길,서울특별시 중구,127.015582,37.566224
2,부동산업,백제고분로48길,송파구,방이동,방이동백제고분로48길,서울특별시 송파구,127.115758,37.510815
3,"전문, 과학 및 기술 서비스업",백제고분로48길,송파구,방이동,방이동백제고분로48길,서울특별시 송파구,127.115758,37.510815
4,도매 및 소매업,백제고분로48길,송파구,방이동,방이동백제고분로48길,서울특별시 송파구,127.115758,37.510815


In [111]:
thezone.columns = seoul_bigvalue_geo_data.columns
thezone.shape

(11966, 8)

In [112]:
fin_data = pd.concat([thezone, seoul_bigvalue_geo_data], axis=0)
fin_data.head()

,업종,ADDR,구,동,동도로명,서울시구,lon,lat
0,도매 및 소매업,퇴계로75길,중구,흥인동,흥인동퇴계로75길,서울특별시 중구,127.016,37.5662
1,"전문, 과학 및 기술 서비스업",퇴계로75길,중구,흥인동,흥인동퇴계로75길,서울특별시 중구,127.016,37.5662
2,부동산업,백제고분로48길,송파구,방이동,방이동백제고분로48길,서울특별시 송파구,127.116,37.5108
3,"전문, 과학 및 기술 서비스업",백제고분로48길,송파구,방이동,방이동백제고분로48길,서울특별시 송파구,127.116,37.5108
4,도매 및 소매업,백제고분로48길,송파구,방이동,방이동백제고분로48길,서울특별시 송파구,127.116,37.5108


In [113]:
# 서울_더존_빅벨류_업종별_동별_경위도.csv로 저장

# fin_data.to_csv('서울_더존_빅벨류_업종별_동별_경위도.csv', encoding='utf-8-sig')

In [116]:
fin_data.shape

(12734, 8)